## Import Needed Libraries

In [1]:
import pandas as pd
import numpy as np

## Import Dataset

In [2]:
df_dogs_info = pd.read_csv('georgia_dogs 09:01.csv')
df_dogs = df_dogs_info.copy()

## Cleaning Data

### Drop Unneeded Columns

In [3]:
# Defining a list of columns that aren't needed for the analysis 
columns_to_drop = ['pet_name', 'secondary_breed', 'primary_colour', 'secondary_colour', 'coat_length', 'children', 'cats', 'other_dogs', 'other_animals', 'fee_waived', 'characteristics', 'house_trained', 'health', 'size', 'shelter_id', 'zip_code']

df_dogs = df_dogs.drop(columns=columns_to_drop)


### Value Consistency

#### Adoption Fee

In [4]:
# Looking at unique adoption fees to identify different structures 
print(df_dogs['adoption_fee'].unique())

['$100.00 ' nan '300' '350' '$200.00 ' '$175.00 ' '$75.00 ' '$250.00 ' '.'
 '$225.00 ' '$195.00 ' '$275.00 ' '$300.00 ' '$50.00 ' '$85.00 '
 '$350.00 ' '$250.01 ' '$125.00 ' '$150.00 ' '$500.00 ' '275' '$450.00 '
 '$270.00 ' '$375.00 ' '$400.00 ' '$25.00 ' '$251.00 ' '$90.00 '
 '$180.00 ' '50' '$298.00 ' '$249.00 ' '$40.00 ' '$325.00 ' '$475.00 '
 '$201.00 ' '$99.00 ' '$49.00 ' '$130.00 ' '$172.00 ' '99' '400'
 '$140.00 ' '$248.00 ' '$199.00 ' '$600.00 ' '450' '$60.00 ' '250' '500'
 '$324.00 ']


In [5]:
# Using regular expression to make consistent structure
df_dogs['adoption_fee'] = df_dogs['adoption_fee'].replace(['N/A', '.'], np.nan)
df_dogs['adoption_fee'] = df_dogs['adoption_fee'].replace({r'\$': ''}, regex=True)

# Converting to numeric coerce errors to NaN
df_dogs['adoption_fee'] = pd.to_numeric(df_dogs['adoption_fee'], errors='coerce')

# Rounding and converting to Int64
df_dogs['adoption_fee'] = df_dogs['adoption_fee'].round().astype('Int64')

# Rounding to nearest 5 (from experience working with shelters, dogs are priced as multiples of 5)
df_dogs['adoption_fee'] = df_dogs['adoption_fee'].apply(lambda x: 5 * round(x / 5) if pd.notnull(x) else x)

In [6]:
print(df_dogs['adoption_fee'].unique())

[100.  nan 300. 350. 200. 175.  75. 250. 225. 195. 275.  50.  85. 125.
 150. 500. 450. 270. 375. 400.  25.  90. 180.  40. 325. 475. 130. 170.
 140. 600.  60.]


#### Pet Location

In [7]:
# Looking at unique pet_locations to identify different structures 
print(df_dogs['pet_location'].unique())

['Fayetteville, GA' 'Peachtree City, GA' 'Snellville, GA' 'Monticello, GA'
 'Jonesboro, GA' 'Atlanta, GA' 'East Point, GA' 'Monroe, GA'
 'Phenix City, AL' 'Dunwoody, GA' 'Decatur, GA' 'Augusta, GA'
 'Lawrenceville, GA' 'Centerville, GA' 'Covington, GA'
 'Peachtree City , GA' 'Macon, GA' 'Locust Grove, GA' 'Marshallville, GA'
 'Stockbridge, GA' 'Milledgeville, GA' 'Albany, GA' 'Perry, GA'
 'Stone Mountain, GA' 'Forsyth, GA' 'Griffin, GA' 'Eastman, GA'
 'Newnan, GA' 'Dawson, GA' 'Norwood, GA' 'Tyrone, GA' 'Washington, GA'
 'Columbus, GA' 'Auburn, GA' 'Winder, GA' 'Peachtree Corners, GA'
 'Madison, GA' 'Colbert, GA' 'Buford, GA' 'Loganville, GA' 'Chamblee, GA'
 'Sharpsburg, GA' 'Cusseta, GA' 'Athens, GA' 'Smyrna, GA'
 'Social Circle, GA' 'ATLANTA , GA' 'WARNER ROBINS, GA' 'Douglas, GA'
 'Grovetown, GA' 'Greensboro, GA' 'Jeffersonville, GA' 'Hamilton, GA'
 'Conyers, GA' 'Danielsville, GA' 'Appling, GA' 'Knoxville, GA'
 'Sparta, GA' 'Milner, GA' 'Douglasville, GA' 'Martinez, GA' 'thomson, G

Findings:
- There are different cases being used 
- Some cities have a space before the comma seperating the city and state
- There are also some cities in Alabama that are present in the pet location, as the shelter might have a partner shelter in Alabama

In [8]:
# Dropping pets in Alabama 
df_dogs = df_dogs[~df_dogs['pet_location'].str.contains(r',\s*AL$', regex=True)]

In [9]:
# Standardizing the pet_location format 
df_dogs['pet_location'] = (df_dogs['pet_location'].str.replace(r'\s*,\s*', ', ', regex=True).str.replace(r'\s+', ' ', regex=True).str.title().str.replace(r'(\bGa\b)', 'GA', regex=True))

In [10]:
print(df_dogs['pet_location'].unique())

['Fayetteville, GA' 'Peachtree City, GA' 'Snellville, GA' 'Monticello, GA'
 'Jonesboro, GA' 'Atlanta, GA' 'East Point, GA' 'Monroe, GA'
 'Dunwoody, GA' 'Decatur, GA' 'Augusta, GA' 'Lawrenceville, GA'
 'Centerville, GA' 'Covington, GA' 'Macon, GA' 'Locust Grove, GA'
 'Marshallville, GA' 'Stockbridge, GA' 'Milledgeville, GA' 'Albany, GA'
 'Perry, GA' 'Stone Mountain, GA' 'Forsyth, GA' 'Griffin, GA'
 'Eastman, GA' 'Newnan, GA' 'Dawson, GA' 'Norwood, GA' 'Tyrone, GA'
 'Washington, GA' 'Columbus, GA' 'Auburn, GA' 'Winder, GA'
 'Peachtree Corners, GA' 'Madison, GA' 'Colbert, GA' 'Buford, GA'
 'Loganville, GA' 'Chamblee, GA' 'Sharpsburg, GA' 'Cusseta, GA'
 'Athens, GA' 'Smyrna, GA' 'Social Circle, GA' 'Warner Robins, GA'
 'Douglas, GA' 'Grovetown, GA' 'Greensboro, GA' 'Jeffersonville, GA'
 'Hamilton, GA' 'Conyers, GA' 'Danielsville, GA' 'Appling, GA'
 'Knoxville, GA' 'Sparta, GA' 'Milner, GA' 'Douglasville, GA'
 'Martinez, GA' 'Thomson, GA' 'Sylvester, GA' 'Jackson, GA' 'Bishop, GA'
 'Newborn

#### Primary Breed

In [11]:
# Looking at unique primary breeds to identify different structures 
print(df_dogs['primary_breed'].unique())

['Basset Hound' 'American Staffordshire Terrier' 'Rat Terrier' 'Chihuahua'
 'Shepherd' 'Australian Shepherd' 'Feist' 'Retriever' 'Labrador Retriever'
 'Maltese' 'Staffordshire Bull Terrier' 'Pointer' 'Terrier'
 'Pit Bull Terrier' 'Australian Cattle Dog / Blue Heeler'
 'American Bulldog' 'Miniature Pinscher' 'Catahoula Leopard Dog'
 'Jack Russell Terrier' 'Poodle' 'Akita' 'Mixed Breed' 'Bull Terrier'
 'German Shepherd Dog' 'Boxer' 'German Pinscher' 'Black Mouth Cur'
 'Cane Corso' 'Yorkshire Terrier' 'Siberian Husky' 'Beagle' 'Tosa Inu'
 'Presa Canario' 'Hound' 'Bloodhound' 'Plott Hound' 'Husky'
 'Wirehaired Terrier' 'English Bulldog' 'Dutch Shepherd'
 'Doberman Pinscher' 'Collie' 'Neapolitan Mastiff' 'Mastiff'
 'Black Labrador Retriever' 'Pug' 'Great Dane'
 'Patterdale Terrier / Fell Terrier' 'Weimaraner' 'Shar-Pei'
 'Yellow Labrador Retriever' 'Fox Terrier' 'Boerboel' 'Bullmastiff'
 'Chocolate Labrador Retriever' 'Golden Retriever' 'Great Pyrenees'
 'West Highland White Terrier / Westi

Findings: 
- It appears that some dogs have two breeds listed as their primary breed, seperated by a "/". In many of these cases, it looks like the breed after the "/" in another name for the breed before the "/"
- In these cases, I am just going to take the breed before the "/"

In [12]:
df_dogs['primary_breed'] = df_dogs['primary_breed'].str.split('/').str[0].str.strip()


### Missing Values

In [13]:
# Replacing blanks with nan to identify missing values 
df_dogs = df_dogs.replace([r'^\s*$', "N/A"], np.nan, regex=True)

col_names = df_dogs.columns.tolist()
dogs_missing_count = df_dogs[col_names].isna().sum()
dogs_missing_share = ((df_dogs[col_names].isna().sum()/df_dogs.shape[0])*100).round(0).astype(int)
dogs_missing_table = pd.DataFrame({
    'Missing Count': dogs_missing_count,
    'Missing Share (%)': dogs_missing_share
})

dogs_missing_table.reset_index(inplace=True)
dogs_missing_table.rename(columns={'index': 'Variable'}, inplace=True)

In [14]:
display(dogs_missing_table)

,Variable,Missing Count,Missing Share (%)
0,pet_id,0,0
1,primary_breed,0,0
2,mixed_breed,0,0
3,age,0,0
4,gender,0,0
5,shelter_name,0,0
6,num_photos,0,0
7,pet_location,0,0
8,adoption_fee,3678,86


Findings:
- The adoption_fee field is missing 86% of values

From experience, I know that most shelters charge a flat rate for each dog based on their age. I will use this knowledge to fill in the missing values based on age for adoption_fee for shelters who list the adoption_fee.  

In [15]:
# Grouping by shelter_name and age, and calculating the mean of adoption_fee while ignoring NaNs
average_adoption_fee = df_dogs.groupby(['shelter_name', 'age'], as_index=False)['adoption_fee'].mean()

print(average_adoption_fee)

                                shelter_name     age  adoption_fee
0                  Joyful Pets Animal Rescue   Adult           NaN
1                  Joyful Pets Animal Rescue    Baby           NaN
2                  Joyful Pets Animal Rescue  Senior           NaN
3                  Joyful Pets Animal Rescue   Young           NaN
4               A Welcome Home Animal Rescue   Adult           NaN
..                                       ...     ...           ...
353  Washington Wilkes Humane Animal Shelter  Senior           NaN
354  Washington Wilkes Humane Animal Shelter   Young           NaN
355                            Whiskers Fund   Adult           NaN
356                     You Lucky Dog Rescue   Adult         300.0
357                     You Lucky Dog Rescue   Young         300.0

[358 rows x 3 columns]


In [16]:
# Merging df_dogs with average_adoption_fee on shelter_name and age
df_dogs = df_dogs.merge(average_adoption_fee, on=['shelter_name', 'age'], suffixes=('', '_avg'))


In [17]:
# Filling nan values in adoption_fee with the calculated average for each shelter-age group
df_dogs['adoption_fee'] = df_dogs['adoption_fee'].fillna(df_dogs['adoption_fee_avg'])

# Dropping the temporary average column 
df_dogs = df_dogs.drop(columns=['adoption_fee_avg'])

In [18]:
# Recomputing missing value count and proportion of adoption_fee
adoption_fee_missing_count = df_dogs['adoption_fee'].isna().sum()
adoption_fee_missing_share = ((df_dogs['adoption_fee'].isna().sum()/df_dogs.shape[0])*100).round(0).astype(int)

print("There are ", adoption_fee_missing_count, " still missing, which is ",  adoption_fee_missing_share, "% of the data.")

There are  3440  still missing, which is  81 % of the data.


Using this method, I was only able to recover an extra 5% of the missing values. 

### Correct Data Format

In [19]:
df_dogs.dtypes

pet_id             int64
primary_breed     object
mixed_breed         bool
age               object
gender            object
shelter_name      object
num_photos         int64
pet_location      object
adoption_fee     float64
dtype: object

Findings:
- The fields are in the correct format 

## Save Cleaned Data to CSV

In [20]:
df_dogs.to_csv('df_dogs_cleaned.csv')